# Simple Analysis of the 2025 NBA Trade Deadline

In [1]:
import numpy as np
import pandas as pd

Let's get a better understanding of who got traded during the deadline, a read of all the packages that got moved, and a working understanding of how the dynamics of this NBA season may have shifted. Here are the following steps that I plan to take.
1. Pull all the trade data (Anything from start of season to Feb 6 2025)
2. Clean it up in a way that's easy to read. Then visualize the nodes teams) and the assets moved (edges)
3. See what each team's WAR delta may have changed.

In [3]:
trades = pd.read_csv("trades.csv")
trades

,Date,Team1,Team2,Team3,Team4,Team5
Dec 15,2024,Warriors: Dennis Schröder,Nets: De’Anthony Melton; Reece Beekman,NaN,NaN,NaN
Dec 15,2024,Pacers: Thomas Bryant,Heat: Draft Compensation,NaN,NaN,NaN
Dec 29,2024,Lakers: Dorian Finney-Smith; Shake Milton,Nets: D’Angelo Russell; Maxwell Lewis,NaN,NaN,NaN
Jan 15,2025,Suns: Nick Richards,Hornets: Josh Okogie,NaN,NaN,NaN
Feb 1,2025,Clippers: Drew Eubanks; Patty Mills,Jazz: Mo Bamba; P.J. Tucker,NaN,NaN,NaN
Feb 2,2025,Lakers: Luka Dončić; Maxi Kleber; Markieff Morris,Mavericks: Anthony Davis; Max Christie,Jazz: Jalen Hood-Schifino,NaN,NaN
Feb 2,2025,Spurs: De’Aaron Fox; Jordan McLaughlin,Kings: Zach LaVine; Sidy Cissoko,Bulls: Zach Collins; Kevin Huerter; Tre Jones,NaN,NaN
Feb 4,2025,Mavericks: Caleb Martin,76ers: Quentin Grimes,NaN,NaN,NaN
Feb 5,2025,Kings: Jonas Valančiūnas,Wizards: Sidy Cissoko,NaN,NaN,NaN
Feb 5,2025,Thunder: Daniel Theis,Pelicans: Draft Compensation,NaN,NaN,NaN
